In [1]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyl

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader

import transformers as T

from peft import get_peft_model, LoraConfig

In [3]:
#@title  { display-mode: "form" }
DATA_NAME = "STS-B"         #@param ["RTE", "STS-B"] {type:"string"}
MODEL_NAME = "roberta-base" #@param  {type:"string"}
PEFT_TYPE = "lora"        #@param ["lora", "bitfit", "full-finetune"] {type:"string"}
RANDOM_SEED = 42            #@param  {type:"integer"}

In [4]:
# 對資料集產生對應的下載網址
filename = f"https://dl.fbaipublicfiles.com/glue/data/{DATA_NAME}.zip"

# 解壓縮後的檔名
entry =  DATA_NAME

# splits -> 資料集的分割名稱
splits = ["train", "dev"]

# 設定執行環境 (CPU or GPU)
device = "cpu" if torch.cuda.is_available() else "cuda"
dataset_dict = dict()

In [5]:
# 確認是否已下載資料
if not os.path.isfile(f"{DATA_NAME}.zip"):
    # 下載資料集
    os.system(f"wget {filename}")
    # 解壓縮
    os.system(f"unzip {DATA_NAME}.zip")

In [6]:
# 讀取所有 splits 資料
for split_type in splits:
    dataset_dict[split_type] = pd.read_csv(
        os.path.join(entry, (split_type + ".tsv")),
        sep="\t", on_bad_lines='skip'
    )

In [7]:
class CustomedDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = { key: torch.tensor(val[idx]) for key, val in self.encodings.items() }
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def process_dataset(df, tokenizer):

    df = df.dropna(ignore_index=True)

    texts = []
    labels = None

    for s1, s2 in zip( df["sentence1"], df["sentence2"] ):
        s1 = s1.strip()
        s2 = s2.strip()
        texts.append(f"<s> {s1} </s></s> {s2} </s>")


    labels = df["score"]

    encodings = tokenizer(
        texts, truncation=True, padding=True, add_special_tokens=False
    )
    return {
        "encodings": encodings,
        "labels": labels.astype(np.float32)
    }

In [8]:
num_labels = 1 # 定義不同資料集的label數量(1 -> regression -> MSELoss)

# 讀入 huggingface 的 model 與 tokenizer
model = T.AutoModelForSequenceClassification.from_pretrained( MODEL_NAME, num_labels=num_labels )
tokenizer = T.AutoTokenizer.from_pretrained( MODEL_NAME )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
# 輸出資料集數量
print("training size:  ", len(dataset_dict["train"]))
print("evaluation size:", len(dataset_dict["dev"]))

training size:   5706
evaluation size: 1465


In [10]:
# 從 dataframe 轉換到 torch dataset
train_dataset = CustomedDataset( **process_dataset(dataset_dict["train"], tokenizer) )
dev_dataset = CustomedDataset( **process_dataset(dataset_dict["dev"], tokenizer) )

In [11]:
from sklearn.metrics import accuracy_score
from scipy.stats import spearmanr

def compute_metrics(pred):
    # 請參考GLUE benchmark的官方網頁，使用和資料集對應的evaluation matrics

    labels = pred.label_ids
    preds = None

    if DATA_NAME != "STS-B":
        preds = pred.predictions.argmax(-1)
    else:
        preds = pred.predictions[:, 0]

    return {"accuracy": accuracy_score(labels, preds)} if DATA_NAME != "STS-B" else {"pearson": spearmanr(labels, preds).statistic}


In [12]:
#@title  { display-mode: "form" }
lora_rank       = 8      #@param  {type:"integer"}
lora_alpha      = 32     #@param  {type:"number"}
lora_dropout    = 0.1    #@param  {type:"number"}

In [13]:
lora_config = {
    "r": lora_rank,
    "lora_alpha": lora_alpha,
    "lora_dropout": lora_dropout
}

In [14]:
def show_trainable_ratio(model):
    trainbale_params = model.num_parameters(only_trainable=True)
    all_params = model.num_parameters(only_trainable=False)
    return f"trainable params: {trainbale_params:,} || all params: {all_params:,} || trainable%: {trainbale_params/all_params*100:.6f}"

In [15]:
if PEFT_TYPE == "lora":
    # LoRA -> 採用 peft 套件的設定
    model = get_peft_model(model, LoraConfig(**lora_config))

elif PEFT_TYPE == "bitfit":
    # 凍結非 bias 的參數
    for name, param in model.named_parameters():
        if "bias" not in name:
            param.requires_grad = False
else:
    # full finetune 不用做任何改動
    pass

print(show_trainable_ratio(model))

trainable params: 294,912 || all params: 124,941,313 || trainable%: 0.236040


In [16]:
#@title  { display-mode: "form" }
num_train_epochs                = 10         #@param  {type:"integer"}
learning_rate                   = 2e-5      #@param  {type:"number"}
per_device_train_batch_size     = 16        #@param  {type:"integer"}
per_device_eval_batch_size      = 16        #@param  {type:"integer"}
gradient_accumulation_steps     = 1         #@param  {type:"integer"}
warmup_steps                    = 200      #@param  {type:"integer"}
weight_decay                    = 0.1      #@param  {type:"number"}
evaluation_strategy             = "steps"   #@param ["steps", "no", "epoch"] {type:"string"}
save_strategy                   = "steps"   #@param ["steps", "no", "epoch"] {type:"string"}
save_steps                      = 150      #@param  {type:"integer"}
eval_steps                      = 150      #@param  {type:"integer"}
adam_epsilon                    = 1e-6      #@param  {type:"number"}
save_total_limit                = 2         #@param  {type:"integer"}
logging_steps                   = 150      #@param  {type:"integer"}

In [17]:
training_args = {
    "output_dir"                    :   DATA_NAME,                  # 設定輸出位置
    "seed"                          :   RANDOM_SEED,                # 設定亂數種子
    "report_to"                     :   "none",
    "label_names"                   :   ["labels"],
    "num_train_epochs"              :   num_train_epochs,
    "learning_rate"                 :   learning_rate,
    "per_device_train_batch_size"   :   per_device_train_batch_size,
    "per_device_eval_batch_size"    :   per_device_eval_batch_size,
    "gradient_accumulation_steps"   :   gradient_accumulation_steps,
    "warmup_steps"                  :   warmup_steps,
    "weight_decay"                  :   weight_decay,
    "adam_epsilon"                  :   adam_epsilon,
    "evaluation_strategy"           :   evaluation_strategy,
    "save_strategy"                 :   save_strategy,
    "save_steps"                    :   save_steps,
    "eval_steps"                    :   eval_steps,
    "save_total_limit"              :   save_total_limit,
    "logging_steps"                 :   logging_steps
}

In [18]:
trainer = T.Trainer(
    model=model,                                        # 🤗 的模型
    args=T.TrainingArguments(**training_args),          # Trainer 所需要的引數
    train_dataset=train_dataset,                        # 訓練集 (注意是 PyTorch Dataset)
    eval_dataset=dev_dataset,                           # 驗證集 (注意是 PyTorch Dataset)，可使 Trainer 在進行訓練時也進行驗證
    compute_metrics=compute_metrics,                    # 自定的評估的指標
)

# 指定使用 1 個 GPU 進行訓練
trainer.args._n_gpu=1

# 開始進行模型訓練
trainer.train()

Step,Training Loss,Validation Loss,Pearson
150,9.245700,7.347072,0.016816
300,5.963800,2.207106,0.329976
450,2.312000,2.284168,0.666480
600,1.872900,1.221138,0.770128
750,1.476600,0.987868,0.779615
900,1.227000,0.871277,0.811273
1050,1.186300,0.836036,0.812194
1200,1.056900,0.822249,0.819190
1350,1.077600,0.772313,0.832088
1500,0.997100,0.790832,0.823294


TrainOutput(global_step=3570, training_loss=1.5509919431029247, metrics={'train_runtime': 3831.889, 'train_samples_per_second': 14.883, 'train_steps_per_second': 0.932, 'total_flos': 1.343933122763898e+16, 'train_loss': 1.5509919431029247, 'epoch': 10.0})

In [19]:
del model, tokenizer
torch.cuda.empty_cache()